# L'inférence de type

---

> Le compilateur de rust est doué de la capacité d'inférer les types des variables. C'est-à-dire qu'il tente de deviner par lui-même de quel type sont les variables que nous utilisons. Ainsi, cela permet au programmeur de ne pas avoir à systématiquement déclarer ses types.

> Pour déterminer automatiquement les types de nos variables, Rust se sert des prototypes des fonctions, des méthodes, des closures et des autres variables avec lesquelles elles interagissent.

> Enfin, s'il n'y a aucune indication de type, Rust peut procéder à une inférence par défaut.

## Les cas d'inférence

### l'Inférence grâce aux fonctions

- Dans l'exemple ci-dessous, il ne sera pas bien difficile à Rust de deviner les types de nos variables d'entrée et de sortie :

In [9]:
{
    fn add_usize(a: usize, b: usize) -> usize {
        a + b
    }
    let a = 1;
    let b = 2;
    let c  = add_usize(a, b);
    dbg!(c);
}

[src/lib.rs:40] c = 3


()

> C'est trivial, a, b et c sont tous les trois des usize.

### l'Inférence grâce aux méthodes

- Une méthode étant seulement une fonction avec self, il n'y a pas de raison que ce soit plus compliqué pour le moteur d'inférence :

In [8]:
{
    struct Tuple(u8, u8, u8);
    impl Tuple {
        fn get_inner(&self) -> (u8, u8, u8) {
            (self.0, self.1, self.2)
        }
    }
    let tuple_struct = Tuple(1, 2, 4);
    let tuple = tuple_struct.get_inner();
    dbg!(tuple);
}

[src/lib.rs:42] tuple = (
    1,
    4,
    2,
)


()

> La variable tuple sera de type (u8, u8, u8)

### L'inférence par l'interaction

- Si j'additionne un usize avec un type non déclaré, quelle déduction fera donc le moteur d'inférence ?

In [13]:
{
    let u: usize = 12;
    let v = u * 2;
    let w = v * v;
    let x = v + 2;
    let y = v - v + 1;
    let z = y * y;
    dbg!(z);
}

[src/lib.rs:148] z = 1


()

> Simple, basique, toutes les variables relatives ici seront des usize.

*Nous verrons les closures plus tard dans le cours, inutile d'en parler ici, de toutes façons, c'est la meme chose que pour les fonctions.*

### L'inférence par défaut

- Quand on ne donne aucune indication quant au type de notre variable ou d'un groupe de variables, Rust procédera, s'il peut, a une inférence par défaut en fonction des valeurs données à nos types :

In [5]:
{
    let u = 12;
    let v = u * 2;
    let w = v * v;
    let x = v + 2;
    let y = v - v + 1;
    let z = y * y;
    dbg!(z); // z est un entier, peut etre isize...
}

[src/lib.rs:29] z = 1


()

*Je ne connais pas de moyen parfait pour connaître le nom exact du type, le compilateur Rust me donne **Integer** qui n'est pas vraiment un type. Si vous trouvez la solution, envoyez-moi un mail, merci.*

## Les conflits de type

- Évidemment, si le développeur fait n'importe quoi avec ses types, certains messages d'erreurs pourront le surprendre :

In [16]:
{
    fn get_u8() -> u8 {
        255
    }
    fn get_u16() -> u16 {
        0
    }
    let a = get_u8();  // type u8
    let b = get_u16(); // type u16
    let c = a * b;
 }

Error: mismatched types

Error: cannot multiply `u8` by `u16`

> Ce n'est pas forcément intuitif puisque comme les types ont été tous inférés, le programmeur ne sait peut être pas trop que son premier type était un u16 et son second un u8. Donc, parfois, c'est le moteur d'inférence qui lui indique une certaine incohérence de son code.

**NB** Rust étant un langage fortement typé, il interdit les opérations algébriques entre deux types différents. Caster l'un deux peut être une solution quoique pas forcément toujours idéale.

## Les limitations de l'inférence de type

> Nous avons déjà remarqué dans le chapitre sur les Option et les Result qu'il existait des types génériques, généralement annotés par des lettres majuscules telles T ou E écrits entre `< >`. Un type générique étant justement un type dont la particularité est de ne pas avoir de type défini à l'avance, le moteur d'inférence de Rust insistera pour que l'on spécifie un type.

- La méthode parse de &str a besoin que l'on définisse F :
```
pub fn parse<F>(&self) -> Result<F, <F as FromStr>::Err>
where
    F: FromStr,
```

In [6]:
{
    // Exemple avec aucun type defini
    let chaine = "1664";
    let entier = chaine.parse().unwrap();
    dbg!(entier);
}

Error: type annotations needed

- Il est nécessaire de donner le type F au compilateur :

In [9]:
{
    let chaine = "1664";
    let entier: u32 = chaine.parse().unwrap(); // entier sera un u32
    dbg!(entier);
}

[src/lib.rs:25] entier = 1664


()

> La lettre F a été remplacée par u32.

**NB prenez comme bonne habitude d'utiliser l'inférence de type, cela ne sert à rien d'avoir dans le code, 50 définitions de type de variable, surtout si le type est identique et que tout semble logique.**